# 匯入套件

In [1]:
from lxml import etree
import requests
import json
import pandas as pd
import jmespath
from datetime import datetime
from bs4 import BeautifulSoup
import numpy as np

# 目標資訊

In [2]:
top5 = "https://fund.api.cnyes.com/fund/api/v2/search/fund?order=return1Month&sort=desc&page=1&institutional=0&isShowTag=1&fundGroup=G1&classCurrency=TWD&categoryAbbr=C46&onshore=1&fields=cnyesId,nav,change,displayNameLocal,lastUpdate,return1Week,return1Month,return3Month,return6Month,returnYTD,return1Year,accReturn2Year,accReturn3Year,accReturn5Year,accReturn10Year"

res = requests.get(top5)
resjson = json.loads(res.text)
result=jmespath.search('items.data[:5].{"基金ID":cnyesId,"最新淨值":nav,"漲跌(%)":change,"基金名稱":displayNameLocal,"更新日期":lastUpdate,"一周績效":return1Week,"一月績效":return1Month,"三月績效":return3Month,"六月績效":return6Month,"今年以來績效":returnYTD,"一年績效":return1Year,"兩年績效":accReturn2Year,"三年績效":accReturn3Year,"五年績效":accReturn5Year,"十年績效":accReturn10Year,"績效表現":[displayNameLocal,cnyesId]}',resjson)
df = pd.DataFrame.from_dict(result)

# 增加api欄位及日期整理

In [3]:
df["更新日期"] = df["更新日期"].apply(lambda x:datetime.fromtimestamp(x).strftime('%Y-%m-%d'))
df["績效表現"] = df["績效表現"].apply(lambda x:f"https://fund.cnyes.com/detail/{str(x[0])}/{str(x[1])}")
df["風險評估"] = df["績效表現"].apply(lambda x:f"{x}/risk")

# 顯示df所有欄位

In [4]:
pd.set_option('display.max_columns', None)#顯示所有col

# info及risk table

In [5]:
def info(link):
    res = requests.get(link)
    soup = BeautifulSoup(res.content,'lxml')
    table = soup.find_all('table')[0] 
    df = pd.read_html(str(table))[0]
    td = soup.select('tr._1DQuA span')
    for j in range(len(td)):
        df.iloc[3][j+1]=str(td[j]).split('data-numerator="')[1].split('"')[0]+'/'+str(td[j]).split('data-denominator="')[1].split('"')[0]
    for k in range(len(td),8):
        df.iloc[3][k+1] = np.nan
    return df

In [6]:
def risk(link):
    res = requests.get(link)
    soup = BeautifulSoup(res.content,'lxml')
    table = soup.find_all('table')[0] 
    df = pd.read_html(str(table))[0]
    return df

# 合併info及risk table

In [7]:
def concatdf(col1,col2,num):
    infodf = info(col1.iloc[num]).set_index('Unnamed: 0')
    riskdf = risk(col2.iloc[num]).rename(columns = {'風險指標':'Unnamed: 0'}).set_index('Unnamed: 0')
    concatdf = infodf.append(riskdf)
    return concatdf

# 新增new colname

In [8]:
while True:
    try:
        tempdf = concatdf(df['績效表現'],df['風險評估'],0)
        if tempdf.index[5] != "夏普值":
            continue
    except IndexError:
        continue
    if tempdf is not None:
        break
for rowidx in range(1,len(tempdf)):
    for colidx in range(len(tempdf.columns)):
        df[str(tempdf.index[rowidx])+"-"+str(tempdf.columns[colidx])] = np.nan

# 填值

In [9]:
for each in range(len(df)):
    while True:
        try:
            tempdf = concatdf(df['績效表現'],df['風險評估'],each)
            if tempdf.index[5] != "夏普值":
                continue
        except IndexError:
            pass
        if tempdf is not None:
            break
    for col in df.columns[17:]:
        df[col].iloc[each] = tempdf.loc[tempdf.index.str.startswith(col.split("-")[0]),[col.split("-")[1]]].values[0][0]
    tempdf is None

C:\Users\user\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [10]:
df

,基金ID,最新淨值,漲跌(%),基金名稱,更新日期,一周績效,一月績效,三月績效,六月績效,今年以來績效,一年績效,兩年績效,三年績效,五年績效,十年績效,績效表現,風險評估,基準指數-1月,基準指數-3月,基準指數-6月,基準指數-今年以來,基準指數-1年,基準指數-3年,基準指數-5年,基準指數-10年,同組平均-1月,同組平均-3月,同組平均-6月,同組平均-今年以來,同組平均-1年,同組平均-3年,同組平均-5年,同組平均-10年,同組排名-1月,同組排名-3月,同組排名-6月,同組排名-今年以來,同組排名-1年,同組排名-3年,同組排名-5年,同組排名-10年,贏過N%基金-1月,贏過N%基金-3月,贏過N%基金-6月,贏過N%基金-今年以來,贏過N%基金-1年,贏過N%基金-3年,贏過N%基金-5年,贏過N%基金-10年,夏普值-1月,夏普值-3月,夏普值-6月,夏普值-今年以來,夏普值-1年,夏普值-3年,夏普值-5年,夏普值-10年,Beta值-1月,Beta值-3月,Beta值-6月,Beta值-今年以來,Beta值-1年,Beta值-3年,Beta值-5年,Beta值-10年,r square-1月,r square-3月,r square-6月,r square-今年以來,r square-1年,r square-3年,r square-5年,r square-10年,標準差-1月,標準差-3月,標準差-6月,標準差-今年以來,標準差-1年,標準差-3年,標準差-5年,標準差-10年,Alpha-1月,Alpha-3月,Alpha-6月,Alpha-今年以來,Alpha-1年,Alpha-3年,Alpha-5年,Alpha-10年
0,A14010,27.87,0.51,新光創新科技基金,2021-04-29,8.73976,19.87097,43.14330,66.58697,43.51184,110.81694,73.86530,51.27391,61.45476,120.93015,https://fund.cnyes.com/detail/新光創新科技基金/A14010,https://fund.cnyes.com/detail/新光創新科技基金/A14010/...,4.84%,6.45%,24.85%,8.01%,52.55%,98.23%,183.39%,428.94%,5.56%,4.59%,27.39%,8.61%,61.48%,81.20%,166.94%,290.23%,1/156,1/152,1/137,1/149,1/125,78/94,71/74,55/57,99%,99%,99%,99%,99%,17%,4%,4%,NaN,NaN,NaN,NaN,3.83,0.67,0.63,NaN,NaN,NaN,NaN,NaN,0.48,0.78,0.76,NaN,NaN,NaN,NaN,NaN,--,0.00,--,NaN,NaN,NaN,NaN,NaN,20.66%,23.94%,20.25%,NaN,NaN,NaN,NaN,NaN,--,-3.22,--,NaN
1,A1SsohV,15.38,0.09,合庫台灣高科技基金,2021-04-29,8.08152,18.12596,24.83766,55.66802,33.85553,NaN,NaN,NaN,NaN,NaN,https://fund.cnyes.com/detail/合庫台灣高科技基金/A1SsohV,https://fund.cnyes.com/detail/合庫台灣高科技基金/A1Ssoh...,4.84%,6.45%,24.85%,8.01%,--,--,--,--,5.56%,4.59%,27.39%,8.61%,61.48%,81.20%,166.94%,290.23%,2/156,8/152,2/137,3/149,NaN,NaN,NaN,NaN,99%,95%,99%,98%,--%,--%,--%,--%,NaN,NaN,NaN,NaN,--,--,--,NaN,NaN,NaN,NaN,NaN,--,--,--,NaN,NaN,NaN,NaN,NaN,--,0.00,--,NaN,NaN,NaN,NaN,NaN,--%,--%,--%,NaN,NaN,NaN,NaN,NaN,--,0.00,--,NaN
2,A33001,24.20,0.18,聯邦精選科技基金,2021-04-29,7.50777,16.45813,26.90089,45.43269,29.75871,74.10072,63.23185,58.52921,148.33732,188.01652,https://fund.cnyes.com/detail/聯邦精選科技基金/A33001,https://fund.cnyes.com/detail/聯邦精選科技基金/A33001/...,4.84%,6.45%,24.85%,8.01%,52.55%,98.23%,183.39%,428.94%,5.56%,4.59%,27.39%,8.61%,61.48%,81.20%,166.94%,290.23%,3/156,3/152,14/137,5/149,22/125,66/94,46/74,46/57,98%,98%,90%,97%,82%,30%,38%,19%,NaN,NaN,NaN,NaN,2.86,0.72,1.04,NaN,NaN,NaN,NaN,NaN,0.49,0.72,0.70,NaN,NaN,NaN,NaN,NaN,--,0.00,--,NaN,NaN,NaN,NaN,NaN,21.83%,24.46%,20.78%,NaN,NaN,NaN,NaN,NaN,--,0.19,--,NaN
3,A19013,48.27,0.66,兆豐國際電子基金,2021-04-29,7.55348,13.30986,27.93533,49.02748,34.19516,77.85556,66.87165,64.84432,114.41977,139.88959,https://fund.cnyes.com/detail/兆豐國際電子基金/A19013,https://fund.cnyes.com/detail/兆豐國際電子基金/A19013/...,4.84%,6.45%,24.85%,8.01%,52.55%,98.23%,183.39%,428.94%,5.56%,4.59%,27.39%,8.61%,61.48%,81.20%,166.94%,290.23%,4/156,2/152,7/137,2/149,19/125,63/94,61/74,52/57,97%,99%,95%,99%,85%,33%,18%,9%,NaN,NaN,NaN,NaN,3.50,0.81,0.95,NaN,NaN,NaN,NaN,NaN,0.36,0.72,0.70,NaN,NaN,NaN,NaN,NaN,--,0.00,--,NaN,NaN,NaN,NaN,NaN,19.36%,23.21%,19.41%,NaN,NaN,NaN,NaN,NaN,--,1.10,--,NaN
4,A11004,107.30,0.93,摩根新興科技基金,2021-04-29,8.39479,12.47379,17.89913,48.65614,23.77437,82.73161,79.41121,59.33387,125.55399,169.44514,https://fund.cnyes.com/detail/摩根新興科技基金/A11004,https://fund.cnyes.com/detail/摩根新興科技基金/A11004/...,4.84%,6.45%,24.85%,8.01%,52.55%,98.23%,183.39%,428.94%,5.56%,4.59%,27.39%,8.61%,61.48%,81.20%,166.94%,290.23%,5/156,14/152,9/137,15/149,13/125,65/94,56/74,48/57,97%,91%,93%,90%,90%,31%,24%,16%,NaN,NaN,NaN,NaN,2.96,0.72,0.94,NaN,NaN,NaN,NaN,NaN,0.59,0.82,0.78,NaN,NaN,NaN,NaN,NaN,--,34.64,--,NaN,NaN,NaN,NaN,NaN,24.49%,24.98%,20.89%,NaN,NaN,NaN,NaN,NaN,--,-2.23,--,NaN


In [12]:
df.to_csv('基金0501.csv',encoding = 'utf-8')